In [1]:
from keras.models import load_model
import cv2
import numpy as np

# Desativa a notação científica
np.set_printoptions(suppress=True)

# Carrega o modelo e rótulos
try:
    modelo = load_model("keras_model.h5", compile=False)
    with open("labels.txt", "r") as arquivo_labels:
        nomes_classes = [linha.strip() for linha in arquivo_labels.readlines()]
except Exception as e:
    print(f"Erro ao carregar modelo ou rótulos: {e}")
    exit()

# Configuração da câmera
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("Erro ao abrir a câmera")
    exit()

# Carrega o classificador Haar Cascade
cascata_face = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Tamanho esperado pelo modelo (normalmente 224x224 para Teachable Machine)
TAMANHO_IMAGEM = (224, 224)

while True:
    ret, imagem = camera.read()
    if not ret:
        print("Erro ao capturar imagem")
        break

    # Conversão para escala de cinza para detecção de faces
    cinza = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = cascata_face.detectMultiScale(cinza, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extrai e redimensiona a região do rosto
        regiao_rosto = imagem[y:y+h, x:x+w]
        regiao_rosto = cv2.resize(regiao_rosto, TAMANHO_IMAGEM)
        
        # Pré-processamento específico para o Teachable Machine
        regiao_rosto = cv2.cvtColor(regiao_rosto, cv2.COLOR_BGR2RGB)  # Convertendo para RGB
        array_rosto = np.asarray(regiao_rosto, dtype=np.float32)
        array_rosto = array_rosto.reshape(1, 224, 224, 3)
        
        # Normalização (ajuste conforme o usado no treinamento)
        array_rosto = (array_rosto / 127.5) - 1  # Normalização para [-1, 1]

        # Faz a previsão
        try:
            previsao = modelo.predict(array_rosto)
            indice = np.argmax(previsao)
            nome_classe = nomes_classes[indice]
            confianca = previsao[0][indice]
            
            # Exibe resultados
            cv2.rectangle(imagem, (x, y), (x+w, y+h), (0, 255, 0), 2)
            texto = f"{nome_classe} ({confianca*100:.2f}%)"
            cv2.putText(imagem, texto, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            print(f"Detectado: {texto}")
        except Exception as e:
            print(f"Erro na previsão: {e}")

    # Exibe a imagem
    imagem_redimensionada = cv2.resize(imagem, (600, 400))
    cv2.imshow("Detecção de Faces - UFOPA", imagem_redimensionada)

    # Verifica se o usuário pressionou ESC
    if cv2.waitKey(1) == 27:
        break

# Libera recursos
camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 729ms/step
Detectado: 2 thor (87.17%)
1/1 [==============================] - 0s 30ms/step
Detectado: 2 thor (92.81%)
1/1 [==============================] - 0s 30ms/step
Detectado: 2 thor (96.74%)
1/1 [==============================] - 0s 32ms/step
Detectado: 2 thor (98.04%)
1/1 [==============================] - 0s 29ms/step
Detectado: 2 thor (94.39%)
1/1 [==============================] - 0s 32ms/step
Detectado: 2 thor (95.96%)
1/1 [==============================] - 0s 34ms/step
Detectado: 2 thor (95.20%)
1/1 [==============================] - 0s 31ms/step
Detectado: 2 thor (95.82%)
1/1 [==============================] - 0s 31ms/step
Detectado: 2 thor (96.96%)
1/1 [==============================] - 0s 44ms/step
Detectado: 2 thor (96.12%)
1/1 [==============================] - 0s 36ms/step
Detectado: 2 thor (96.20%)
1/1 [==============================] - 0s 36ms/step
Detectado: 2 thor (94.22%)
1/1 [==============================] - 0s 32ms/step